In [38]:
import json
import pandas as pd
import numpy as np
from pathlib import Path

In [39]:
year = None

In [56]:
def filt_dfs(xls, sheet, year):
    df = xls.parse(sheet)
    cl = df.iloc[12, 13]
    nl = df.iloc[12, 4]
    print(f"{cl},", nl)
    if not year:
        year = df.iloc[7, 23].replace("/", "-")
    df = df.iloc[16:, 2:].dropna(axis=1, how='all')
    # first_name,last_name,email,student_id,current_class,gender,date_of_birth
    df.columns = ["student_id", "last_name", "first_name", "gender", "date_of_birth", "Place"]
    df.fillna("", inplace=True)
    df["date_of_birth"] = pd.to_datetime(df["date_of_birth"]).dt.strftime("%Y-%m-%d")
    for c in ["last_name", "first_name", "gender", "Place"]:
        df[c] = df[c].str.title()
        df[c] = df[c].str.strip()
    df = df.drop('Place', axis=1)
    df["current_class"] = cl.replace("--", "-")
    df["email"] = "student@company.com"
    df["gender"] = np.where(df['gender'] == 'Garçon', 'M', 'F')
    df = df[["first_name", "last_name", "email", "student_id", 'current_class', "gender", "date_of_birth"]]
    return df, year

# Step 1: Load the Excel file

In [41]:
xls = pd.ExcelFile("/home/mohamed/Downloads/Files/Xlss/listeleve_202206232francais.xls")
pd.set_option('future.no_silent_downcasting', True)

In [42]:
sheets = xls.sheet_names

In [57]:
result = pd.DataFrame()
seasons = pd.DataFrame()
for sheet in sheets:
    df, year = filt_dfs(xls, sheet, year)
    result = pd.concat([result, df])

TCSF-1, Tronc commun Sciences - Option Français
TCSF-2, Tronc commun Sciences - Option Français
TCSF-3, Tronc commun Sciences - Option Français
1APIC-1, 1ère Année  Collégial Parcours International
1APIC-2, 1ère Année  Collégial Parcours International
1APIC-3, 1ère Année  Collégial Parcours International
1APIC-4, 1ère Année  Collégial Parcours International
1APIC-5, 1ère Année  Collégial Parcours International
1APIC-6, 1ère Année  Collégial Parcours International
1APIC-7, 1ère Année  Collégial Parcours International
1APIC-8, 1ère Année  Collégial Parcours International
1APIC-9, 1ère Année  Collégial Parcours International
2APIC-1, 2ème Année Collégial Parcours International
2APIC-2, 2ème Année Collégial Parcours International
2APIC-3, 2ème Année Collégial Parcours International
2APIC-4, 2ème Année Collégial Parcours International
2APIC-5, 2ème Année Collégial Parcours International
2APIC-6, 2ème Année Collégial Parcours International
2APIC-7, 2ème Année Collégial Parcours International

In [58]:
result.to_csv("/home/mohamed/Downloads/result.csv",index=False)

In [52]:
result = result.reset_index(drop=True).set_index('Nbr').to_dict(orient='index')

KeyError: "None of ['Nbr'] are in the columns"

In [55]:
result["D134778663"]

{'Nom': 'Dergham',
 'Prenom': 'Alal',
 'Genre': 'Garçon',
 'Birthdate': '2007-09-09',
 'Place': '',
 '2021-2022': '1APIC-5'}

In [56]:
with Path("/home/mohamed/Documents/datas/Docker/Students.json").open("w") as fl:
    json.dump(result, fl)